# Projeto THLOP/SAM - Demonstração

**Laboratório SPEED**<br> 
**Departamento de Ciência da Computação**<br>
**Universidade Federal de Minas Gerais**<br>

Julho, 2023

***

Esta apresentação tem como objetivo ilustrar algumas das principais contribuições do projeto até o momento. Algumas delas, já abordadas no artigo *Caracterização Escalável de Vulnerabilidades de Segurança: um Estudo de Caso na Internet Brasileira*, no SBRC 2022 (DOI: https://doi.org/10.5753/sbrc.2022.222341).

O caso de uso desta apresentação, "Um estudo das 16 vulnerabilidades mais exploradas de 2021", detalhado a seguir,  embora seja um exemplo específico contêm um conjunto de operações que poderiam ser utilizadas pelo CERT de forma geral. Por exemplo: 

* Reformatação da base do Shodan e processamento paralelo no Spark;
* Integração com datasets externos (e.g., dados do NVD e CVE para completar as informações do Shodan e bases de cidades para geolocalização);
* Processamento dos dados para limpeza e padronização (e.g., nome de organizações);
* Utilização de funções e operações comuns ao processamento e análises de dados de redes como os do Shodan, de forma amigável e encapsulada;


## Caso de uso: Um estudo das 16 vulnerabilidades mais exploradas de 2021

Segundo uma matéria do site [CISO Advisor](https://www.cisoadvisor.com.br/as-30-vulnerabilidades-mais-exploradas-de-2020-e-2021/), um total de 16 vulnerabilidades para as quais já existe correção continuam sendo amplamente exploradas no mundo inteiro porque estão em dispositivos não atuaizados pelos seus proprietários. A lista publicada foi criada a partir de um comunicado conjunto publicado na quarta-feira dia 28 de julho, o FBI e a Cybersecurity and Infrastructure Security Agency (CISA), mais o Australian Cyber Security Center e o National Cyber Security Center do Reino Unido listaram essas vulnerabilidades mais exploradas no primeiro semestre de 2021. Segundo o comunicado, os cibercriminosos continuam a explorar as vulnerabilidades de software publicamente conhecidas – e frequentemente antigas – contra amplos conjuntos de alvos, incluindo organizações dos setores público e privado em todo o mundo.

A lista de 2021 é a seguinte:

- Microsoft Exchange: CVE-2021-26855, CVE-2021-26857, CVE-2021-26858 e CVE2021-27065
- Pulse Secure: CVE-2021-22893, CVE-2021-22894, CVE-2021-22899 e CVE-2021-22900
- Accellion: CVE-2021-27101, CVE-2021-27102, CVE-2021-27103, CVE-2021-27104
- VMware: CVE-2021-21985
- Fortinet: CVE-2018-13379, CVE-2020-12812 e CVE-2019-5591


Sobre isso, queremos responder perguntas como:

- Qual o grau de vulnerabilidade desses CVES? Algum desses foram encontrados no Brasil pela amostra do Shodan? 
- Como é a distribuição dessas vulnerabilidades entre as organizações no Brasil ?

### 0. Importação das bibliotecas, inicialização do Spark e leitura da base do Shodan

In [ ]:
import os                                                         
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 

from tlhop.shodan_abstraction import DataFrame  # Extensão da API de DataFrame do Spark
from tlhop.datasets import DataSets  # API de importaçao de base externas

INPUT = "XXXXX"

# optional settings
TMP_PORT = os.environ["SPARK_UI_PORT"]
SPARK_TMP_PATH = os.environ["SPARK_TMP_PATH"]

spark = (
    SparkSession.builder.master("local[10]")
        .config("spark.driver.memory", "40g") 
        .config("spark.local.dir", SPARK_TMP_PATH) # optional settings to force Spark to use a different UI port
        .config("spark.ui.port", TMP_PORT) # optional settings to force Spark to use a different temporary folder
        .getOrCreate()
        )

df = spark.read.format("delta").load(INPUT) # Leitura do dados do Shodan (já convertidos via ShodanDatasetManager)

### 1. Iniciando a API de base de dados externa e listando as bases de dados disponíveis

In [ ]:
ds = DataSets() # inicalizando nossa API de bases externas
ds.list_datasets() # listando os datasets disponíveis atualmente

### 2. Carregando a base de vulnerabilidades do NIST.

Essa base poderá nos dar maiores detalhes sobre detalhes sobre os CVEs informados na matéria.

In [ ]:
nvd_df = ds.read_dataset("NVD_CVE_LIB") # carregando a base de vulnerabilidades do NIST
nvd_df.printSchema() # exibindo o esquema de dados desse DataFrame

### 2.1 Filtrando as vulnerabilidades da lista para obter informações dos CVEs

Como podemos encontrar os CVEs descritos na matéria na base de dados do NIST? Uma das formas é utilizando nossa extensão da API de DataFrame do Spark, que forneçe um conjunto de operadores de alto nível, mais específicas ao contexto de análises em dados como os do Shodan. 

Nossa API, estendida pelo operador `shodan_extension`, implementa funções complexas, como:
 - `contains_vulnerability()`:  Verifica se um CVE está presente em um banner;
 - `cleaning_org()`: Extrai a organização vinculada ao banner e padroniza-a.

In [ ]:
cves = ["CVE-2021-26855", "CVE-2021-26857", "CVE-2021-26858", "CVE2021-27065", "CVE-2021-22893", "CVE-2021-22894", 
        "CVE-2021-22899", "CVE-2021-22900", "CVE-2021-27101", "CVE-2021-27102", "CVE-2021-27103", "CVE-2021-27104", 
        "CVE-2021-21985", "CVE-2018-13379", "CVE-2020-12812", "CVE-2019-5591"]

nvd_df_filtered = (
        nvd_df.shodan_extension.contains_vulnerability(cves, mode="any", vulns_col="cve_id") # filtrando os cves no NIST
            .select("cve_id", "cvssv3", "publishedDate", "rank_cvss_v3") # projeção de colunas
            .orderBy(desc("cvssv3"), asc("cve_id")) # ordenando resultado por <CVSS (v3), CVE>
        )

nvd_df_filtered.show()

### 3. Essas vulnerabilidades foram encontradas na internet brasileira ?

Usando a amostra de dados do Shodan (jan - jul de 2021), podemos novamente executar o `contains_vulnerability` para procurar *banners* com esses CVEs. Quantos *banners* foram encontrados no total?

In [ ]:
# contando os banners com pelo menos um cve da lista
df.shodan_extension.contains_vulnerability(cves, mode="any").count()

Foram encontrados 10494 banners com tais vulnerabilidades!

### 3.1. E se quisermos saber quais vulnerabilidades e sua frequência foram? 

Uma dificuldade dessa tarefa é que um *banner* pode conter 0 ou N vulnerabilidades (ou seja, a coluna `vulns_cve` é uma lista de strings), logo, não basta identificar se um *banner* possui as vulnerabilidades da lista. Precisamos transofrmar o registro para conter uma vulnerabilidade por banner (função `explode`). 

Depois disso, podemos realizar uma junção com a tabela do NIST (que agora contém apenas os CVEs da mátéria) para filtrar os CVEs de alvo para a computação.

In [ ]:
(
    df.withColumn("vulns_cve", explode("vulns_cve")) # quebrando a lista de vulns em um CVE por registro
        .join(nvd_df_filtered, col("vulns_cve") == col("cve_id")) # junção com o NVD/NIST para filtrar apenas os cves desejados
        .groupby("vulns_cve", "cvssv3", "publishedDate", "rank_cvss_v3").count() # contabilizando as ocorrências de cada cve
        .orderBy(desc("vulns_cve")) # ordenando o resultado por ordem descrescente
        .show()
)

Foram encontrados os CVEs 'CVE-2021-26855', 'CVE-2021-26857' e 'CVE-2021-26858' (relacionadas entre si). Essas vulnerabilidades permitem a execução remota de código do Microsoft Exchange Server.

### 4. Quantas organizações possuem tais vulnerabilidades ?

Para uma melhor qualidade do resultado, podemos executar o nosso operador `cleaning_org` para limpeza e padronização dos nomes das organizações. 

In [ ]:
orgs = (
    df.shodan_extension.contains_vulnerability(cves, mode="any") # filtrando banners com pelo menos um cve da lista
        .filter(col("org").isNotNull())  # filtrar banners com informação de organização
        .shodan_extension.cleaning_org(input_col="org", output_col="org_clean")  # padronização do campo de organização
        .groupby("org_clean").count()  # Contabilização a quantidade de banners com as vulnerabilidades para cada organização
        .orderBy(desc("count"))  # ordenação descrescente
)
orgs.count()

### 4.1. Como podemos computar o PDF(Função densidade) ou CDF (Função de distribuição cumulativa) da frequência dessas organizações?  

Para isso, podemos usar o nosso operador `gen_cdf_pdf`, que recebe uma coluna que representa uma frequência de valores para a computação da PDF/CDF.

In [ ]:
orgs_list = (
    df.shodan_extension.contains_vulnerability(cves, mode="any") # filtrando banners com pelo menos um cve da lista
        .shodan_extension.cleaning_org(input_col="org", output_col="org_clean")  # padronização do campo de organização
        .shodan_extension.gen_cdf_pdf("org_clean", to_pandas=True) # Contabiliza a quantidade de banners com as 
          # vulnerabilidades para cada organização e gera a CDF/PDF da sua distribuição, salvando o resultado como Pandas
          # para o plot
)
orgs_list.plot(y=["org_clean_pdf"], figsize=(10, 5))

A lista abaixo apresenta a frequência (anonimizada) das top 5 organizações com mais banners identificadas com as vulnerabilidades alvo:

In [ ]:
orgs_list[["frequency", "org_clean_pdf"]][0:5]

### 5. Georreferenciamento

Nessa última parte, vamos querer relacionar as vulnerabilidades e organizações as cidades brasileirias. Para isso, vamos precisar de 2 etapas: 

1. Recomputar a lista de organizações/vulnerabilidades, relacionando suas cidades de atuação;
2. Incluir informações da base externa de cidades brasileiras para um melhor georrefeciamento; 

### 5.1. Gerando lista das organizações por cidade

Procedimento igual ao feito na etapa 4, apenas com adição do campo `city`;

In [ ]:
step1 = (
    df.shodan_extension.contains_vulnerability(cves, mode="any") # filtrando banners com pelo menos um cve da lista
        .filter(col("org").isNotNull())  # filtrar banners com informação de organização
        .shodan_extension.cleaning_org()  # padronização do campo de organização
        .filter(col("city").isNotNull())  # filtrar banners com informação de cidade
        .groupby("org_clean", "city").count()  # Contabilização da tupla <organizacao, cidade>
)

### 5.2. Incoporando e mesclando a base das cidades brasileiras na relação de organizações.

Utilizando a base de cidades brasileiras, podemos estender as informações providas pelo Shodan com outras diversas informações como: informação se a cidade é capital, número de habitantes, UF, dados censitários, entre outras. No nosso caso, utilizamos essa base no exemplo para uma melhor definição do campo de Latitude e Longitude. Embora o Shodan já forneça essas duas informações, a informação não é confiável.

No código a seguir, realizamos a inclusão dessa base para a computação de quantas organizações distintas foram identificadas em cada cidade e qual a soma de banners coletados pelo Shodan nessa cidade, a partir dessas organizações.

In [ ]:
brazil_cities = ds.read_dataset("BRAZILIAN_CITIES")\
    .select("CITY", "STATE", "CAPITAL", "LAT", "LONG")  # projeção de campos de interesse

step2 = (
    step1.join(brazil_cities, step1["city"] == brazil_cities["CITY"])          # junção com a base de cidades brasileiras
        .groupby(step1["city"], brazil_cities["LAT"], brazil_cities["LONG"])   # agrupando os dados pela cidade
        .agg(countDistinct("org_clean").alias("n_orgs"),                       # e computando o número de organizações distintas
             sum("count").alias("n_banners"))                                  # junto com a quantidade de banners no total
)
step2.show()

### 5.3. Gerando gráfico de bolha da distribuição das organizações pelas cidades brasileiras (tamanho e cor representam a quantidade de organizações distintas na região)

In [ ]:
import plotly
plotly.offline.init_notebook_mode() # only to make plotly save the vizualiation on the notebook

fig = step2.shodan_extension.plot_bubble(lat_col="LAT", 
                                         lon_col="LONG", 
                                         color_col="n_orgs", 
                                         size="n_orgs", 
                                         hover_name="city", 
                                         hover_data=["n_orgs","n_banners"], 
                                         opacity=0.8)
fig.update_layout(autosize=False, width=2000, height=500)

### 8.4. Gerando gráfico de densidade Heatmap da quantidade de banners por região (cores mais claras representam regiões com alta densidade de banners)

In [ ]:
fig = step2.shodan_extension.plot_heatmap(lat_col="LAT", 
                                          lon_col="LONG", 
                                          z_col="n_banners",  
                                          hover_name="city", 
                                          hover_data=["n_orgs","n_banners"], 
                                          radius=20)
fig.update_layout(autosize=False, width=2000, height=500)

In [ ]:
spark.stop()